In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv


In [2]:
!pip install qiskit==0.43.1 qiskit-machine-learning==0.6.1


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 18.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 80.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 21.0 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 8.6 MB/s 

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report

from qiskit.circuit.library import ZZFeatureMap, TwoLocal
from qiskit.algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_machine_learning.algorithms.classifiers import VQC


In [4]:
df = pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.dropna(subset=['TotalCharges'], inplace=True)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
df = pd.get_dummies(df, columns=['Contract'])

quantum_features = [
    'tenure', 'MonthlyCharges', 'TotalCharges', 'SeniorCitizen',
    'Contract_One year', 'Contract_Two year'
]

scaler = MinMaxScaler()
df[quantum_features] = scaler.fit_transform(df[quantum_features])
df_quantum = df[quantum_features + ['Churn']]


In [5]:
X = df_quantum.drop('Churn', axis=1).to_numpy()
y = df_quantum['Churn'].to_numpy()

# Keep it small for quantum speed
X_small, _, y_small, _ = train_test_split(X, y, train_size=200, stratify=y, random_state=42)


In [6]:
num_qubits = X_small.shape[1]
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=1)
ansatz = TwoLocal(num_qubits, ['ry', 'rz'], 'cz', reps=1)
optimizer = COBYLA(maxiter=100)
sampler = Sampler()

vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    sampler=sampler
)

vqc.fit(X_small, y_small)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = vqc.predict(X_small)

print("✅ Quantum Classifier Accuracy:", accuracy_score(y_small, y_pred))
print(classification_report(y_small, y_pred))


✅ Quantum Classifier Accuracy: 0.68
              precision    recall  f1-score   support

           0       0.83      0.71      0.76       147
           1       0.43      0.60      0.50        53

    accuracy                           0.68       200
   macro avg       0.63      0.66      0.63       200
weighted avg       0.72      0.68      0.69       200



In [8]:
# Step 1: Select 1000 samples
from sklearn.model_selection import train_test_split

X_opt, _, y_opt, _ = train_test_split(X, y, train_size=1000, stratify=y, random_state=42)

# Step 2: Define feature map and ansatz
from qiskit.circuit.library import ZZFeatureMap, TwoLocal

num_qubits = X_opt.shape[1]

feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=2)
ansatz = TwoLocal(num_qubits, ['ry', 'rz'], 'cz', reps=2)

# Step 3: Optimizer and sampler
from qiskit.algorithms.optimizers import SPSA
from qiskit.primitives import Sampler
from qiskit_machine_learning.algorithms.classifiers import VQC

optimizer = SPSA(maxiter=300)
sampler = Sampler()

# Step 4: Build and train VQC
vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    sampler=sampler
)

# ⏱ Optional: Track training time
import time
start_time = time.time()

vqc.fit(X_opt, y_opt)

end_time = time.time()
print(f"✅ VQC trained on 1000 samples in {end_time - start_time:.2f} seconds")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


✅ VQC trained on 1000 samples in 14148.82 seconds


In [9]:
from sklearn.metrics import accuracy_score, classification_report

y_pred_opt = vqc.predict(X_opt)

print("✅ Optimized VQC Accuracy:", accuracy_score(y_opt, y_pred_opt))
print(classification_report(y_opt, y_pred_opt))


✅ Optimized VQC Accuracy: 0.628
              precision    recall  f1-score   support

           0       0.85      0.60      0.70       734
           1       0.39      0.70      0.50       266

    accuracy                           0.63      1000
   macro avg       0.62      0.65      0.60      1000
weighted avg       0.73      0.63      0.65      1000

